# 데이터 이상치 분석 자동화 실험

## 필요한 모듈 설치

In [ ]:
%pip install langchain-ollama=0.2.0
%pip install langchain=0.3.4
%pip install matplotlib=3.9.2
%pip install numpy=1.26.4
%pip install pandas=2.2.3
%pip install pyarrow=17.0.0
%pip install seaborn=0.13.2
%pip install sentencepiece=0.2.0
%pip install transformers=4.45.2

In [7]:
import pandas as pd
import pyarrow as pa
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from transformers import AutoTokenizer

## 데이터 로드

In [ ]:
file_path = '../data/train_dataset/train/dataset.arrow'
with pa.memory_map(file_path, 'r') as source:
    table = pa.ipc.open_stream(source).read_all()
df: pd.DataFrame = table.to_pandas()
df.head()

In [3]:
text_column = 'context'  # 예: 'context', 'document' 등
if text_column not in df.columns:
    print(f"Warning: '{text_column}' column not found. Please specify the correct column name.")

\* 로컬 Ollama 서버로 llama2 활용하는 방법 (macOS 기준)
1. Ollama 설치
    ```bash
    brew install ollama
    ```
2. Ollama 서버 실행
    ```bash
    ollama run llama2
    ```
3. Ollama 서버 중단
    ```bash
    pkill ollama
    ```

## 모델 로드 및 프롬프트 세팅

In [4]:
llm = OllamaLLM(model="llama2")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b")

Tree of Thoughts 적용 프롬프트

In [5]:
text_analysis_template = """
You are a Data Analyst. You are tasked with doing EDA for modeling Open-Domain Question Answering. You are tasked with analyzing the following text based on potential anomalies and preprocessing requirements. Use a Tree of Thoughts approach to evaluate multiple reasoning paths. Answer the following questions strictly with 'Yes' or 'No' where applicable, and provide concise reasoning or recommendations if an issue is detected.

Context: {context},
Question: {question},
Answer: {answer}

### Step 1: Anomaly Detection
1. Does this text contain any structural, semantic, or formatting anomalies? (Yes/No)
    - If 'Yes', explain the anomaly briefly.

### Step 2: Special Character Evaluation
2. Are there any unnecessary special characters or symbols in this text that do not contribute to the meaning? (Yes/No)
    - If 'Yes', specify which characters should be removed or replaced.

### Step 3: Preprocessing Requirements
3. Does this text require any preprocessing to improve its structure or readability? (Yes/No)
    - If 'Yes', specify the type of preprocessing required (e.g., punctuation removal, spacing correction, formatting adjustments).

### Step 4: Sufficiency of Context
4. Is the provided context sufficient and specific enough to answer the given question correctly? (Yes/No)
    - If 'No', explain briefly why the context is inadequate.

### Step 5: Logical Consistency
5. Can a human logically infer the answer to the given question from the provided context? (Yes/No)
    - If 'No', explain the logical defection briefly.

Answer each question concisely based on your findings. Only answer with 'Yes' when you found ANY OUTLYING. Answer 'No' ONLY IF ALL CONDITIONS MATCH.
"""

text_analysis_prompt = PromptTemplate(
    input_variables=["context", "question", "answer"],
    template=text_analysis_template
)

text_analysis_chain = (
    {
        "context": RunnablePassthrough(),
        "question": RunnablePassthrough(),
        "answer": RunnablePassthrough(),
    }
    | text_analysis_prompt
    | llm  # Executes the analysis through Llama2 or your chosen LLM
    | StrOutputParser()  # Parses the result into a string format
)

In [ ]:
MAX_TOKEN_LIMIT = 2048
TEMPLATE_OVERHEAD = 150

Llama2 입력 전 제어를 위한 코드

In [ ]:
def trim_input(context, question, answer, template_length=TEMPLATE_OVERHEAD):
    """
    Ensures that the combined input stays within the token limit.
    If the context is too long, it will be truncated to fit.
    """
    # Tokenize the context, question, and answer using LlamaTokenizer
    context_tokens = tokenizer.encode(context, add_special_tokens=False)
    question_tokens = tokenizer.encode(question, add_special_tokens=False)
    answer_tokens = tokenizer.encode(answer, add_special_tokens=False)

    # Calculate how much space is left for the context after accounting for the question, answer, and template
    available_tokens_for_context = MAX_TOKEN_LIMIT - (len(question_tokens) + len(answer_tokens) + template_length)

    # If context exceeds the available space, truncate it
    if len(context_tokens) > available_tokens_for_context:
        # Truncate the context tokens
        context_tokens = context_tokens[:available_tokens_for_context]

    # Decode the truncated context back into text
    trimmed_context = tokenizer.decode(context_tokens, clean_up_tokenization_spaces=True)
    
    return trimmed_context


QA 데이터를 LLM에 입력하고 응답을 반환하는 코드

In [ ]:
def process_qa_sample(context, question, answer):
    # Trim the context if necessary
    trimmed_context = trim_input(context, question, answer)

    # Invoke the LLM analysis with the trimmed context
    result = text_analysis_chain.invoke({
        "context": trimmed_context,
        "question": question,
        "answer": answer
    })

    return result

In [ ]:
context = df.iloc[696]["context"]
question = df.iloc[696]["question"]
answer = df.iloc[696]["answer"]
result_txt = process_qa_sample(context, question, answer)

for i, c in enumerate(result_txt):
    if i % 50 == 0:
        print(f"{c}\n")
    else:
        print(f"{c}", end="")

In [6]:
def analyze_special_chars(text):
    # Detect special characters that are not common in ODQA data
    special_chars = re.findall(r'[^\w\s\.\,\!\?\"\'\:\;\-\(\)\[\]\{\}]', text)
    return list(set(special_chars))

데이터에서 결측치를 감지하는 코드

In [7]:
def detect_simple_anomalies(text, question, answer):
    """Detects simple anomalies in text, question, or answer."""
    special_chars = analyze_special_chars(text)
    is_text_missing = text is None or text.strip() == ""
    is_question_missing = question is None or question.strip() == ""
    is_answer_missing = answer is None or answer.strip() == ""

    simple_anomalies = []

    if special_chars:
        simple_anomalies.append(f"Special characters: {special_chars}")
    
    if is_text_missing:
        simple_anomalies.append("Missing or empty context.")
    if is_question_missing:
        simple_anomalies.append("Missing or empty question.")
    if is_answer_missing:
        simple_anomalies.append("Missing or empty answer.")

    return simple_anomalies


In [8]:
# Set the sample size to either 100 or the size of the DataFrame, whichever is smaller
sample_size = min(100, len(df))
sample_indices = np.random.choice(len(df), sample_size, replace=False)

# Open the file for writing the results
with open('autoEDA.txt', 'w') as f:
    for idx in sample_indices:
        # Extract the text from the context column
        text = df.iloc[idx]['context']
        question = df.iloc[idx]['question']
        answer = df.iloc[idx]['answers']['text'][0]  # Assuming first answer in list

        # Detect simple anomalies
        simple_anomalies = detect_simple_anomalies(text, question, answer)

        # Run LLM analysis
        result = text_analysis_chain.invoke(
            {
                "context": text,
                "question": question,
                "answer": answer,
            }
        )

        # If any anomalies detected, write them to the file
        if simple_anomalies or ('Yes' in result):
            f.write(f"Sample {idx}:\n")
            f.write(f"Context: {text[:100]}...\n" if len(text) > 100 else f"Context: {text}\n")
            f.write(f"Question: {question}\n")
            f.write(f"Answer: {answer}\n")
            
            if simple_anomalies:
                f.write("Simple Anomalies:\n")
                f.write("\n".join(simple_anomalies) + "\n")
            
            if 'Yes' in result:
                f.write("LLM Analysis:\n")
                f.write(f"{result}\n")
            
            f.write("-" * 50 + "\n")
